In [ ]:
import numpy as np
import pandas as pd

from tqdm.auto import tqdm

import utils
from utils import evaluate_keep

In [ ]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
from sklearn import metrics

from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler, ADASYN, BorderlineSMOTE

In [ ]:
RANDOM_SEED = 744

In [ ]:
shavers = utils.Shavers('<path_to_your_dataset>',
                     normalize_transform=utils.normalize_resnet18)
X = utils.get_resnet18_embeddings(shavers)

y = np.array(shavers.targets)
class_to_idx = shavers.class_to_idx

In [ ]:
val, cnt = np.unique(y, return_counts=True)
i2c = {i:c for c, i in class_to_idx.items()}

val = [i2c[x] for x in val]
for v, c in zip(val, cnt):
    print(v, ":", c, round(c / len(y), 3))

In [ ]:
all_results = pd.DataFrame()
KEEP_RATIO = [1.0, 0.75, 0.5, 0.25]

### Include DRAEM anomaly maps

Generate anomaly maps for your dataset using **DRAEM-anomaly_maps.ipynb**

In [ ]:
draem_heatmap = utils.Shavers('<path_to_anomaly_maps>',
                     normalize_transform=utils.normalize_resnet18)
X_heatmap = utils.get_resnet18_embeddings(draem_heatmap)
y_heatmap = np.array(draem_heatmap.targets)
assert all(y_heatmap == y)

## Experiments

In [ ]:
X_comb = np.hstack([X, X_heatmap])

In [ ]:
# Baseline performance
mlp = MLPClassifier(random_state=RANDOM_SEED, max_iter=10000)
all_results = all_results.append(evaluate_keep(mlp, X, y, 'baseline'))

In [ ]:
# Baseline performance + DRAEM
mlp = MLPClassifier(random_state=RANDOM_SEED, max_iter=10000)
all_results = all_results.append(evaluate_keep(mlp, X_comb, y, 'baseline+draem'))

In [ ]:
# Only DRAEM
mlp = MLPClassifier(random_state=RANDOM_SEED, max_iter=10000)
all_results = all_results.append(evaluate_keep(mlp, X_heatmap, y, 'baseline-onlydraem'))

In [ ]:
# pip install catboost
from catboost import Pool, CatBoostClassifier
model = CatBoostClassifier(iterations=60, depth=10, loss_function='MultiClass')
r = evaluate_keep(model, X, y, 'catboost')

### Over-sampling

In [ ]:
# Random over-sampling
mlp = MLPClassifier(random_state=RANDOM_SEED, max_iter=10000)
sampler = RandomOverSampler(random_state=RANDOM_SEED)
model = make_pipeline(sampler, mlp)
all_results = all_results.append(evaluate_keep(model, X, y, 'random-oversample'))

In [ ]:
# Random over-sampling
mlp = MLPClassifier(random_state=RANDOM_SEED, max_iter=10000)
sampler = RandomOverSampler(random_state=RANDOM_SEED)
model = make_pipeline(sampler, mlp)
all_results = all_results.append(evaluate_keep(model, X_comb, y, 'random-oversample+draem'))

In [ ]:
m = CatBoostClassifier(iterations=60, depth=10, loss_function='MultiClass')
sampler = RandomOverSampler(random_state=RANDOM_SEED)
model = make_pipeline(sampler, m)
all_results = all_results.append(evaluate_keep(model, X, y, 'random-oversample-catboost'))

In [ ]:
# Random over-sampling
mlp = MLPClassifier(random_state=RANDOM_SEED, max_iter=10000)
sampler = ADASYN(random_state=RANDOM_SEED)
model = make_pipeline(sampler, mlp)
all_results = all_results.append(evaluate_keep(model, X, y, 'adasyn-oversample'))

In [ ]:
# ADASYN over-sampling => catboost
m = CatBoostClassifier(iterations=60, depth=10, loss_function='MultiClass')
sampler = ADASYN(random_state=RANDOM_SEED)
model = make_pipeline(sampler, m)
all_results = all_results.append(evaluate_keep(model, X, y, 'adasyn-oversample-catboost'))

### SMOTE

In [ ]:
k = 2
m = 20
b_k = 2
sampler = BorderlineSMOTE(random_state=RANDOM_SEED, k_neighbors=k, m_neighbors=m, kind=f'borderline-{b_k}')
mlp = MLPClassifier(random_state=RANDOM_SEED, max_iter=10000)
model = make_pipeline(sampler, mlp)
all_results = all_results.append(evaluate_keep(model, X_comb, y, 'smote-oversample+draem'))

In [ ]:
k = 2
m = 20
b_k = 2
sampler = BorderlineSMOTE(random_state=RANDOM_SEED, k_neighbors=k, m_neighbors=m, kind=f'borderline-{b_k}')
m = CatBoostClassifier(iterations=60, depth=10, loss_function='MultiClass')
model = make_pipeline(sampler, m)
all_results = all_results.append(evaluate_keep(model, X, y, 'smote-oversample-catboost'))

### Augmentation with GAN images

In [ ]:
fake = utils.Shavers('../lightweight-gan/generated/',
                     normalize_transform=utils.normalize_resnet18)
X_fake = utils.get_resnet18_embeddings(fake)
y_fake = np.array(fake.targets)

assert fake.class_to_idx == class_to_idx

In [ ]:
rs = []
for keep in tqdm(KEEP_RATIO):
    cv = ReductionStratifiedKFold(n_splits=10, keep=keep, 
                                     good_class=class_to_idx['good'],
                                     random_state=RANDOM_SEED)
    scorer = make_custom_scorer(class_to_idx)
    rng = np.random.RandomState(RANDOM_SEED)
    
    for fold, (train, test) in tqdm(enumerate(cv.split(X, y)), leave=False):
        # Balance the classes with fake images
        lbs, cnts = np.unique(y[train], return_counts=True)
        inds = np.arange(len(y_fake))

        fake_inds = []
        for lb, cnt in zip(lbs, cnts):
            fake_inds.append(
                rng.choice(inds[y_fake == lb], cnts.max() - cnt))

        fake_inds = np.concatenate(fake_inds)
        xtr = np.vstack([X[train], X_fake[fake_inds]])
        ytr = np.concatenate([y[train], y_fake[fake_inds]])
      
        mlp = MLPClassifier(random_state=RANDOM_SEED, max_iter=10000)
        mlp.fit(xtr, ytr)
        
        d = scorer(mlp, X[test], y[test])
        d.update({'model': 'gan', 'keep': keep, 'fold': fold})
        rs.append(d)
    

## Significance tests

In [ ]:
import itertools
import scipy

In [ ]:
column = 'test_binary_recall'

In [ ]:
# Pairwise comparison for all experiments
pairs = []
for m1, r1 in df.groupby('model'):
    for m2, r2 in df.groupby('model'):
        if m1 == m2: continue
        pair = {'a': m1, 'b': m2}
        
        d = r1[column] - r2[column]
        for alt in ['two-sided', 'greater', 'less']:
            r = scipy.stats.wilcoxon(d, alternative=alt)
            pair[alt + '_p'] = r.pvalue
            pair[alt + '_s'] = r.statistic
            
        pairs.append(pair)
pairs = pd.DataFrame(pairs)

In [ ]:
pairs[['a', 'b', 'two-sided_p', 'greater_p', 'less_p']]